In [1]:
with open('/home/phamanh195/Downloads/adventOfCodeDay5.txt', 'rt') as f: raw_input = f.read()

In [14]:
test_input = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

In [3]:
from collections import namedtuple
import re
import sys

Almanac = namedtuple('Almanac', [
    'seeds',
    'seed_soil_map',
    'soil_fertilizer_map', 
    'fertilizer_water_map', 
    'water_light_map', 
    'light_temperature_map', 
    'temperature_humidity_map',
    'humidity_location_map',
])

RangeMapItem = namedtuple('RangeMapItem', ['source', 'destination', 'length'])

class RangeMap:
    def __init__(self, *range_items: list[RangeMapItem]):
        self.ranges = list(range_items)
        self._sort()
    
    def _sort(self):
        self.ranges.sort(key=lambda x: x.source)
    
    def add(self, range_item: RangeMapItem):
        self.ranges.append(range_item)
        self._sort()

    def find(self, item: int) -> int:
        for range_item in self.ranges:
            source, des, length = range_item
            if source <= item <= (source + length - 1):
                delta = item - source
                result = des + delta
                return result
        return item
    
    def reverse(self):
        new_ranges = [
            RangeMapItem(source=des, destination=src, length=length)
            for src, des, length in self.ranges
        ]
        return self.__class__(*new_ranges)

def parse_map(key: str, raw_input: str) -> RangeMap:
    key_re = rf'.*{key} map:\n([0-9\s]*)'
    map_text = re.search(key_re, raw_input).group(1).strip()
    key_map = RangeMap()
    for row in map_text.splitlines():
        des, source, length = list(map(int, row.strip().split()))
        key_map.add(RangeMapItem(source, des, length))
    return key_map

def parse_almanac(raw_input: str) -> Almanac:
    seeds = []
    seed_soil_map = {}
    soil_fertilizer_map = {}
    fertilizer_water_map = {}
    water_light_map = {}
    light_temperature_map = {}
    temperature_humidity_map = {}
    humidity_location_map = {}
    
    # Parse seeds
    seeds_re = r'.*seeds: (.*)\n\n'
    seeds = list(map(int, re.search(seeds_re, raw_input).group(1).split()))
    
    # Parse seed soil
    seed_soil_map = parse_map(key='seed-to-soil', raw_input=raw_input)
    
    # Parse soil fertilizer
    soil_fertilizer_map = parse_map(key='soil-to-fertilizer', raw_input=raw_input)
    
    fertilizer_water_map = parse_map(key='fertilizer-to-water', raw_input=raw_input)
    
    water_light_map = parse_map(key='water-to-light', raw_input=raw_input)
    
    light_temperature_map = parse_map(key='light-to-temperature', raw_input=raw_input)
    
    temperature_humidity_map = parse_map(key='temperature-to-humidity', raw_input=raw_input)
    
    humidity_location_map = parse_map(key='humidity-to-location', raw_input=raw_input)
    
    return Almanac(
        seeds, 
        seed_soil_map,
        soil_fertilizer_map, 
        fertilizer_water_map, 
        water_light_map, 
        light_temperature_map, 
        temperature_humidity_map,
        humidity_location_map,
    )

Chain = namedtuple('Chain', [
    'seed', 'soil', 'fertilizer', 'water', 'light', 'temperature', 'humidity', 'location'
])
    
def trace(almanac: Almanac, seed: int) -> Chain:
    soil = almanac.seed_soil_map.find(seed)
    fertilizer = almanac.soil_fertilizer_map.find(soil)
    water = almanac.fertilizer_water_map.find(fertilizer)
    light = almanac.water_light_map.find(water)
    temperature = almanac.light_temperature_map.find(light)
    humidity = almanac.temperature_humidity_map.find(temperature)
    location = almanac.humidity_location_map.find(humidity)
    return Chain(seed, soil, fertilizer, water, light, temperature, humidity, location)

def lowest_location(raw_input: str) -> int:
    almanac = parse_almanac(raw_input=raw_input)
    chains = [
        trace(almanac=almanac, seed=i) for i in almanac.seeds
    ]
    return min([chain.location for chain in chains])

def ngroup(lst: list, n: int):
    return zip(*(iter(lst),) * n)

ZipRangeItem = namedtuple('ZipRangeItem', ['start', 'length'])


class ZipRange:
    def __init__(self):
        self.ranges: list[ZipRangeItem] = []
    
    def append(self, zip_range: ZipRangeItem):
        self.ranges.append(zip_range)
    
    def __iter__(self):
        for range_item in self.ranges:
            start, length = range_item
            for item in range(start, start + length):
                yield item
    
    def __repr__(self):
        return str(self.ranges)

def parse_almanac_v2(raw_input: str) -> Almanac:
    seeds = []
    seed_soil_map = {}
    soil_fertilizer_map = {}
    fertilizer_water_map = {}
    water_light_map = {}
    light_temperature_map = {}
    temperature_humidity_map = {}
    humidity_location_map = {}
    
    # Parse seeds
    seeds_re = r'.*seeds: (.*)\n\n'
    raw_seed_ranges = list(map(int, re.search(seeds_re, raw_input).group(1).split()))
    seeds = ZipRange()
    for seed_range in ngroup(raw_seed_ranges, 2):
        seeds.append(ZipRangeItem(*seed_range))

    # Parse seed soil
    seed_soil_map = parse_map(key='seed-to-soil', raw_input=raw_input)
    
    # Parse soil fertilizer
    soil_fertilizer_map = parse_map(key='soil-to-fertilizer', raw_input=raw_input)
    
    fertilizer_water_map = parse_map(key='fertilizer-to-water', raw_input=raw_input)
    
    water_light_map = parse_map(key='water-to-light', raw_input=raw_input)
    
    light_temperature_map = parse_map(key='light-to-temperature', raw_input=raw_input)
    
    temperature_humidity_map = parse_map(key='temperature-to-humidity', raw_input=raw_input)
    
    humidity_location_map = parse_map(key='humidity-to-location', raw_input=raw_input)
    
    return Almanac(
        seeds, 
        seed_soil_map,
        soil_fertilizer_map, 
        fertilizer_water_map, 
        water_light_map, 
        light_temperature_map, 
        temperature_humidity_map,
        humidity_location_map,
    )

def lowest_location_v2(raw_input: str) -> int:
    almanac = parse_almanac_v2(raw_input=raw_input)
    change_points = set()
    chain_maps = [
        almanac.seed_soil_map,
        almanac.soil_fertilizer_map,
        almanac.fertilizer_water_map,
        almanac.water_light_map,
        almanac.light_temperature_map,
        almanac.temperature_humidity_map,
        almanac.humidity_location_map,
    ]
    rchain_maps = chain_maps[::-1]
    
    change_points = set()
    for chain_map in rchain_maps:
        tmp_change_points = {
            0, sys.maxsize
        }
        for range_item in chain_map.ranges:
            tmp_change_points |= {
                range_item.source,
                range_item.source + range_item.length - 1,
                range_item.source + range_item.length,
            }
            if range_item.source > 0:
                tmp_change_points.add(range_item.source - 1)
        
        rchain_map = chain_map.reverse()
        tmp_change_points |= {
            rchain_map.find(cp) for cp in change_points
        }
        change_points = tmp_change_points

    check_points = set()
    for seed_range in almanac.seeds.ranges:
        start = seed_range.start
        end = seed_range.start + seed_range.length - 1
        check_points |= {start, end}
        for change_point in change_points:
            if start < change_point < end:
                check_points.add(change_point)
            if start < change_point + 1 < end:
                check_points.add(change_point + 1)
    min_value = float('inf')
    for seed in check_points:
        chain = trace(almanac=almanac, seed=seed)
        min_value = min(min_value, chain.location)
    return min_value

def lowest_location_v2_slow(raw_input: str) -> int:
    almanac = parse_almanac_v2(raw_input=raw_input)
    min_value = float('inf')
    mseed = None
    for seed in almanac.seeds:
        location = trace(almanac, seed).location
        if location < min_value:
            min_value = location
            mseed = seed
    return min_value, mseed

In [4]:
print(lowest_location_v2(raw_input))
print(lowest_location_v2_slow(test_input))

79874951


NameError: name 'test_input' is not defined

In [135]:
# 137568992
# 100165128